# EDA — Real BAPRO Corpus

Exploratory data analysis of the ingested article corpus and FSI target.
Run after `make backfill` and `make seed_fsi` have populated the database.

In [ ]:
# Cell 1 — Setup
import json
import sys
from pathlib import Path

# Allow imports from repo root
repo_root = Path().resolve().parent
if str(repo_root) not in sys.path:
    sys.path.insert(0, str(repo_root))

import numpy as np
import pandas as pd
from sqlalchemy import text

from db.connection import get_engine

engine = get_engine()

# Load articles
with engine.connect() as conn:
    articles_df = pd.read_sql(
        text("SELECT id, date, headline, gdelt_themes, source FROM articles ORDER BY date"),
        conn,
    )
    fsi_df = pd.read_sql(
        text("SELECT date, fsi_value FROM fsi_target ORDER BY date"),
        conn,
    )
    emb_count = conn.execute(text("SELECT COUNT(*) FROM article_embeddings")).scalar()

articles_df["date"] = pd.to_datetime(articles_df["date"])
fsi_df["date"] = pd.to_datetime(fsi_df["date"])

print(f"Articles: {len(articles_df):,}")
print(f"Embeddings: {emb_count:,}")
print(f"FSI rows: {len(fsi_df):,}")
print(f"Date range: {articles_df['date'].min().date()} to {articles_df['date'].max().date()}")
articles_df.head()

In [ ]:
# Cell 2 — Temporal heatmap: articles per week x day-of-week
import matplotlib
matplotlib.use('Agg')  # non-interactive backend for headless execution
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches

docs_dir = Path("../docs")
docs_dir.mkdir(exist_ok=True)

articles_df["week"] = articles_df["date"].dt.isocalendar().week.astype(int)
articles_df["year"] = articles_df["date"].dt.year
articles_df["year_week"] = articles_df["year"].astype(str) + "-W" + articles_df["week"].astype(str).str.zfill(2)
articles_df["dow"] = articles_df["date"].dt.dayofweek  # 0=Mon

pivot = articles_df.groupby(["year_week", "dow"]).size().unstack(fill_value=0)
pivot.columns = ["Mon", "Tue", "Wed", "Thu", "Fri", "Sat", "Sun"][: len(pivot.columns)]

fig, ax = plt.subplots(figsize=(14, max(4, len(pivot) * 0.4)))
im = ax.imshow(pivot.values.T, aspect="auto", cmap="YlOrRd")
ax.set_xticks(range(len(pivot)))
ax.set_xticklabels(pivot.index, rotation=90, fontsize=7)
ax.set_yticks(range(len(pivot.columns)))
ax.set_yticklabels(pivot.columns)
ax.set_title("Articles per week x day-of-week")
plt.colorbar(im, ax=ax, label="article count")
plt.tight_layout()
out_path = docs_dir / "eda_temporal_heatmap.png"
fig.savefig(out_path, dpi=100)
print(f"Saved heatmap to {out_path}")
plt.close(fig)

In [ ]:
# Cell 3 — Token distribution: headline word count histogram
articles_df["word_count"] = articles_df["headline"].fillna("").str.split().str.len()

fig, ax = plt.subplots(figsize=(8, 4))
ax.hist(articles_df["word_count"], bins=40, color="steelblue", edgecolor="white")
ax.set_xlabel("Headline word count")
ax.set_ylabel("Number of articles")
ax.set_title("Headline token distribution")
ax.axvline(articles_df["word_count"].median(), color="red", linestyle="--",
           label=f"Median: {articles_df['word_count'].median():.0f}")
ax.legend()
plt.tight_layout()
out_path2 = docs_dir / "eda_token_distribution.png"
fig.savefig(out_path2, dpi=100)
print(f"Saved token hist to {out_path2}")
print(articles_df["word_count"].describe().round(1).to_string())
plt.close(fig)

In [ ]:
# Cell 4 — Duplicate detection
# Exact duplicates by headline
dup_exact = articles_df[articles_df.duplicated(subset=["headline"], keep=False)]
print(f"Exact headline duplicates: {len(dup_exact)} articles ({len(dup_exact) / max(len(articles_df), 1) * 100:.1f}%)")

# Cosine similarity sample (n=500 random pairs)
SAMPLE_N = min(500, len(articles_df))

with engine.connect() as conn:
    sample_rows = conn.execute(
        text(
            "SELECT ae.id, ae.embedding FROM article_embeddings ae "
            "ORDER BY RANDOM() LIMIT :n"
        ),
        {"n": SAMPLE_N},
    ).fetchall()

if sample_rows:
    vecs = []
    for row in sample_rows:
        emb = row[1]
        vecs.append(json.loads(emb) if isinstance(emb, str) else list(emb))
    vecs = np.array(vecs)

    # Normalise
    norms = np.linalg.norm(vecs, axis=1, keepdims=True)
    norms = np.where(norms == 0, 1, norms)
    vecs_norm = vecs / norms

    # Sample 200 random pairs
    rng = np.random.default_rng(42)
    idx_a = rng.integers(0, len(vecs_norm), 200)
    idx_b = rng.integers(0, len(vecs_norm), 200)
    cosines = np.sum(vecs_norm[idx_a] * vecs_norm[idx_b], axis=1)

    near_dup_threshold = 0.95
    near_dups = np.sum(cosines > near_dup_threshold)
    print(f"Near-duplicate pairs (cosine > {near_dup_threshold}) in sample: {near_dups} / 200")
    print(f"Cosine similarity stats: mean={cosines.mean():.3f}, p95={np.percentile(cosines, 95):.3f}")
else:
    print("No embeddings found in DB. Run embed step first.")

In [ ]:
# Cell 5 — FSI correlation: article volume vs FSI over time (dual-axis chart)
daily_counts = articles_df.groupby("date").size().rename("article_count").reset_index()
merged = pd.merge(daily_counts, fsi_df, on="date", how="inner")

if merged.empty:
    print("No overlap between articles and FSI dates. Check data.")
else:
    fig, ax1 = plt.subplots(figsize=(14, 5))
    ax2 = ax1.twinx()

    ax1.bar(merged["date"], merged["article_count"], color="steelblue", alpha=0.6, label="Article count")
    ax2.plot(merged["date"], merged["fsi_value"], color="crimson", linewidth=2, label="FSI")

    ax1.set_xlabel("Date")
    ax1.set_ylabel("Article count", color="steelblue")
    ax2.set_ylabel("FSI value", color="crimson")
    ax1.set_title("Daily article volume vs Financial Stress Index")

    handles1, labels1 = ax1.get_legend_handles_labels()
    handles2, labels2 = ax2.get_legend_handles_labels()
    ax1.legend(handles1 + handles2, labels1 + labels2, loc="upper left")

    plt.tight_layout()
    out_path3 = docs_dir / "eda_fsi_vs_volume.png"
    fig.savefig(out_path3, dpi=100)
    print(f"Saved FSI chart to {out_path3}")

    corr = merged["article_count"].corr(merged["fsi_value"])
    print(f"Pearson correlation article_count vs fsi_value: {corr:.3f}")
    plt.close(fig)

In [ ]:
# Cell 6 — Summary table with quality gates
gates = [
    ("Total articles",         len(articles_df),  100,    "articles in DB"),
    ("Embeddings coverage",    emb_count,         len(articles_df) * 0.95,  "embeddings >= 95% of articles"),
    ("FSI rows",               len(fsi_df),       50,     "FSI business days"),
    ("Exact dup rate (%)",     round(len(dup_exact) / max(len(articles_df), 1) * 100, 1), None, "informational"),
]

print("\n=== Quality Gate Summary ===")
print(f"{'Metric':<30} {'Value':>10} {'Threshold':>12} {'Pass?':>6}")
print("-" * 62)
all_pass = True
for name, value, threshold, note in gates:
    if threshold is None:
        status = "INFO"
    elif value >= threshold:
        status = "PASS"
    else:
        status = "FAIL"
        all_pass = False
    thresh_str = str(threshold) if threshold is not None else "N/A"
    print(f"{name:<30} {value:>10} {thresh_str:>12} {status:>6}  # {note}")

print()
if all_pass:
    print("All required quality gates PASSED.")
else:
    print("WARNING: Some quality gates FAILED. Check data ingestion.")